In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import joblib
import os
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split

C:\Users\Administrator\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [41]:
gbdt = joblib.load('./gbdt.pkl')

In [2]:
# 数据加载
file_list = []
for root_dir,sub_dir,files in os.walk('./data'):
    file_list.append(files)
file_list = file_list[0]
dataframe_list = pd.DataFrame()
for file in files:
    if file.endswith('xls'):
        file_name = os.path.join(root_dir,file)
        df = pd.read_excel(file_name)
        dataframe_list = dataframe_list.append(df)
dataframe_list.drop(['Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21','Unnamed: 22','Unnamed: 23','创角账号数'],
                    axis=1,inplace=True)
dataframe_list.columns = ['plan_name','channel_id','source_id','data_win','source_run_datetime','create_time','plan_id',
                         'game_id','platform','source_run_date_amount','create_role_num','create_role_cost','create_role_pay_num',
                          'create_role_pay_cost','create_role_pay_sum','create_role_roi','create_role_retain_1d']
dataframe_list.dropna(how='any',inplace=True)

In [4]:
dataframe_list['media_id'] = 10

In [6]:
# 入参
# "columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform",
#      "source_run_date_amount","create_role_num","create_role_cost","create_role_pay_num","create_role_pay_cost",
#             "create_role_pay_sum","create_role_roi","create_role_retain_1d"]

In [7]:
data = dataframe_list[["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform",
     "source_run_date_amount","create_role_num","create_role_cost","create_role_pay_num","create_role_pay_cost",
            "create_role_pay_sum","create_role_roi","create_role_retain_1d"]]

In [8]:
data.head()

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
1,7162.0,299591.0,0.0,2020-05-01,2020-04-24,10,1001049.0,1.0,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.001590,0.0522
2,6770.0,297966.0,0.0,2020-05-01,2020-04-21,10,1001049.0,1.0,0.00,682.0,0.000000,6.0,0.000000,590.0,0.000000,0.0940
3,20754.0,304584.0,0.0,2020-05-01,2020-05-01,10,1001049.0,1.0,6603.77,640.0,10.334538,3.0,2201.256667,66.0,0.009994,0.0516
4,20754.0,304143.0,0.0,2020-05-01,2020-04-30,10,1000840.0,1.0,6640.08,406.0,16.476625,2.0,3320.040000,218.0,0.032831,0.0571
5,20754.0,301128.0,0.0,2020-05-01,2020-04-27,10,1000840.0,1.0,3850.09,270.0,14.474023,3.0,1283.363333,66.0,0.017143,0.0451


In [38]:
data.shape

(1718817, 16)

In [63]:
data.values

array([[7162.0, 299591.0, 0.0, ..., 318.0, 0.00159, 0.0522],
       [6770.0, 297966.0, 0.0, ..., 590.0, 0.0, 0.094],
       [20754.0, 304584.0, 0.0, ..., 66.0, 0.0099943, 0.0516],
       ...,
       [20452.0, 337576.0, 7.0, ..., 0.0, 0.0, 0.0],
       [20452.0, 337578.0, 7.0, ..., 0.0, 0.0, 0.0],
       [20143.0, 337587.0, 7.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [65]:
aa = pd.DataFrame({'A':[1,2,3],'B':[4,5,6]})

In [141]:
aa = data.sample(100000)

In [103]:
aa['create_time'] = aa['create_time'].astype(str)

In [84]:
aa = aa.values.tolist()

In [86]:
file=open('data.txt','w') 
file.write(str(aa)); 
file.close() 

In [111]:
jsondata = {"columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"],
           "data":aa.values.tolist()}

In [107]:
type(jsondata)

dict

In [110]:
jsondata

{'columns': ['channel_id',
  'source_id',
  'data_win',
  'source_run_datetime',
  'create_time',
  'media_id',
  'game_id',
  'platform',
  'source_run_date_amount',
  'create_role_num',
  'create_role_cost',
  'create_role_pay_num',
  'create_role_pay_cost',
  'create_role_pay_sum',
  'create_role_roi',
  'create_role_retain_1d'],
 'data': array([[7162.0, 261134.0, 7.0, ..., 0.0, 0.0, 0.0],
        [6867.0, 294573.0, 5.0, ..., 0.0, 0.0, 0.0],
        [7162.0, 217274.0, 6.0, ..., 0.0, 0.0, 0.0],
        ...,
        [21088.0, 333893.0, 5.0, ..., 0.0, 0.0, 0.0],
        [20754.0, 292204.0, 2.0, ..., 0.0, 0.0, 0.0],
        [20143.0, 314932.0, 2.0, ..., 0.0, 0.0, 0.0]], dtype=object)}

In [117]:
# j = json.dumps(jsondata)

In [116]:
# import json

# with open("tmp.txt", "w") as fp:
#     fp.write(json.dumps(jsondata))

In [112]:
columns_1 = source_predict(jsondata=jsondata)['columns']
data_1 = source_predict(jsondata=jsondata)['list']

In [45]:
result = pd.DataFrame(data=data_1,columns=columns_1)

In [47]:
result.shape

(30194, 17)

In [48]:
result['source_run_date_amount'].min()

500.15

In [49]:
groups = result.groupby(['channel_id','source_id','source_run_datetime'])
groups.ngroups

30194

In [50]:
result['label'].value_counts()

1    27087
0     3107
Name: label, dtype: int64

In [51]:
result[result['label']==0]['data_win'].value_counts()

0.0    1848
3.0     708
7.0     373
2.0     178
Name: data_win, dtype: int64

In [53]:
result[result['label']==0].head()

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,label
7,20954.0,336540.0,0.0,2020-07-30,2020-07-28,10,1000993.0,2.0,22838.39,648.0,35.353545,5.0,4567.678000,158.0,0.006918,0.0867,0
12,20755.0,336867.0,0.0,2020-07-30,2020-07-29,10,1000993.0,2.0,1286.17,46.0,27.960217,1.0,1286.170000,12.0,0.009330,0.0435,0
18,20755.0,307729.0,0.0,2020-05-09,2020-05-09,10,1000993.0,2.0,4528.30,137.0,33.053285,1.0,4528.300000,30.0,0.006625,0.0657,0
22,20954.0,336540.0,0.0,2020-07-29,2020-07-28,10,1000993.0,2.0,42549.78,982.0,43.418143,11.0,3868.161818,700.0,0.016451,0.0918,0
32,20954.0,307422.0,0.0,2020-05-09,2020-05-09,10,1000993.0,2.0,3695.65,98.0,37.710714,1.0,3695.650000,60.0,0.016235,0.0510,0


In [56]:
result.to_csv('./result.csv')

In [90]:
aa.head()

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
15390,20545.0,301828.0,5.0,2020-07-21,2020-04-28,10,1000840.0,1.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0000
18046,20755.0,243404.0,1.0,2020-05-11,2019-12-07,10,1000954.0,2.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0000
41770,20953.0,319112.0,4.0,2020-06-17,2020-06-13,10,1000840.0,1.0,558.25,7.0,79.75,0.0,0.0,0.0,0.0,0.1429
14267,20754.0,332926.0,6.0,2020-07-28,2020-07-19,10,1000840.0,1.0,0.19,1.0,0.19,0.0,0.0,0.0,0.0,0.0000
3088,20754.0,222773.0,2.0,2020-06-06,2019-10-22,10,1000960.0,1.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0000


In [262]:
bb = pd.DataFrame(data=[[1,1,3,"2020-03-30","2020-03-30",10,80024,1,1000,5,100,2,250,20,0.25,0.2],
      [1,1,7,"2020-03-30","2020-03-30",10,80023,2,2000,10,0,4,500,40,0.5,0.2]],columns = ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"])

In [263]:
bb

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
0,1,1,3,2020-03-30,2020-03-30,10,80024,1,1000,5,100,2,250,20,0.25,0.2
1,1,1,7,2020-03-30,2020-03-30,10,80023,2,2000,10,0,4,500,40,0.50,0.2


In [264]:
jsondata = {"columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"],
           "data":bb.values}

In [265]:
columns_1 = source_predict(jsondata)['columns']
data_1 = source_predict(jsondata)['list']

In [266]:
data_1

[[1,
  1,
  3,
  '2020-03-30',
  '2020-03-30',
  10,
  80024,
  1,
  1000,
  5,
  100,
  2,
  250,
  20,
  0.25,
  0.2,
  1]]

In [ ]:
def source_predict(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"]}
    :return:
    '''
    data = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])
    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]    # 删除消耗为0的数据
    # 去重，处理某天没有消耗的情况
    data.sort_values(by='data_win',inplace=True)
    data.drop_duplicates(subset=['channel_id','source_id','source_run_date_amount','create_role_num'],keep='first'
                           ,inplace=True)
    # 对消耗500元以下的不作判断，对数据按平台、win进行分类处理
    data_ignore = data[data['source_run_date_amount'] < 500]
    data_not_ignore = data[data['source_run_date_amount'] >= 500]
    
    data_an = data_not_ignore[data_not_ignore['platform'] == 1]
    data_ios = data_not_ignore[data_not_ignore['platform'] == 2]

    data_win_0 = data_an[data_an['data_win'] == 0]
    data_win_1 = data_an[data_an['data_win'] == 1]
    data_win_2 = data_an[data_an['data_win'] == 2]
    data_win_3 = data_an[data_an['data_win'] == 3]
    data_win_7 = data_an[data_an['data_win'] == 7]

    data_win_0_ios = data_ios[data_ios['data_win'] == 0]
    data_win_1_ios = data_ios[data_ios['data_win'] == 1]
    data_win_2_ios = data_ios[data_ios['data_win'] == 2]
    data_win_3_ios = data_ios[data_ios['data_win'] == 3]
    data_win_7_ios = data_ios[data_ios['data_win'] == 7]
    
    result_df = pd.DataFrame()
    temp_df = pd.DataFrame()
    # win=0预判
    if data_win_0.shape[0] != 0:
        data_win_0['label'] = data_win_0.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_cost>=20)&
            (x.create_role_pay_num==0) else(0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.02) else 
                                            (1 if x.create_role_cost>=50 else 2))),axis=1)
        result_df = result_df.append(data_win_0[(data_win_0['label'] == 1)| (data_win_0['label'] == 0)])
        temp_df = temp_df.append(data_win_0[data_win_0['label'] == 2])
    if data_win_0_ios.shape[0] != 0:    
        data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_cost>=40)&
            (x.create_role_pay_num==0) else(0 if (x.create_role_cost<=300)&(x.create_role_roi>=0.02) else 
                                            (1 if x.create_role_cost>=120 else 2))),axis=1)
        result_df = result_df.append(data_win_0_ios[(data_win_0_ios['label'] == 1)| (data_win_0_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_0_ios[data_win_0_ios['label'] == 2])
        
    # win=1预判
    if data_win_1.shape[0] != 0:
        data_win_1['label'] = data_win_1.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.02)
                                    else (1 if x.create_role_cost>=50 else (1 if x.create_role_pay_sum==0 else 2))),axis=1)
        result_df = result_df.append(data_win_1[(data_win_1['label'] == 1)| (data_win_1['label'] == 0)])
        temp_df = temp_df.append(data_win_1[data_win_1['label'] == 2])
    if data_win_1_ios.shape[0] != 0:
        data_win_1_ios['label'] = data_win_1_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=300)&(x.create_role_roi>=0.02)
                                        else (1 if x.create_role_cost>=120 else (1 if x.create_role_pay_sum==0 else 2))),axis=1)
        result_df = result_df.append(data_win_1_ios[(data_win_1_ios['label'] == 1)| (data_win_1_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_1_ios[data_win_1_ios['label'] == 2])
    
    # win=2预判
    if data_win_2.shape[0] != 0:
        data_win_2['label'] = data_win_2.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.045)
                                    else (1 if x.create_role_cost>=50 else (1 if x.create_role_roi<=0.03 else 2))),axis=1)
        result_df = result_df.append(data_win_2[(data_win_2['label'] == 1)| (data_win_2['label'] == 0)])
        temp_df = temp_df.append(data_win_2[data_win_2['label'] == 2])
        
    if data_win_2_ios.shape[0] != 0:
        data_win_2_ios['label'] = data_win_2_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=250)&(x.create_role_roi>=0.045)
                                        else (1 if x.create_role_cost>=120 else (1 if x.create_role_roi<=0.02 else 2))),axis=1)
        result_df = result_df.append(data_win_2_ios[(data_win_2_ios['label'] == 1)| (data_win_2_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_2_ios[data_win_2_ios['label'] == 2])
   
    # win=3预判
    if data_win_3.shape[0] != 0:
        data_win_3['label'] = data_win_3.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.07)
            else (1 if x.create_role_pay_cost > 5000 else (1 if x.create_role_retain_1d<0.05 else(1 if x.create_role_roi<0.04 else 2))))),axis=1)
        result_df = result_df.append(data_win_3[(data_win_3['label'] == 1)| (data_win_3['label'] == 0)])
        temp_df = temp_df.append(data_win_3[data_win_3['label'] == 2])
    if data_win_3_ios.shape[0] != 0:
        data_win_3_ios['label'] = data_win_3_ios.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (0 if (x.create_role_cost<=250)&(x.create_role_roi>=0.07)
                else (1 if x.create_role_pay_cost >10000 else (1 if x.create_role_retain_1d<0.05 else(1 if x.create_role_roi<0.04 else 2))))),axis=1)
        result_df = result_df.append(data_win_3_ios[(data_win_3_ios['label'] == 1)| (data_win_3_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_3_ios[data_win_3_ios['label'] == 2])
    
    
    # 模型预测
    if temp_df.shape[0] != 0:
        temp_df['label'] = gbdt.predict(temp_df[['data_win','platform','create_role_cost','create_role_pay_cost','create_role_roi']])
    # win=7
    if data_win_7.shape[0] != 0:
        data_win_7['label'] = data_win_7.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=50)&(x.create_role_roi>=0.125)&(x.create_role_pay_cost<=4000) else 1), axis=1)
        result_df = result_df.append(data_win_7[(data_win_7['label'] == 1)| (data_win_7['label'] == 0)])
        temp_df = temp_df.append(data_win_7[data_win_7['label'] == 2])
    if data_win_7_ios.shape[0] != 0:
        data_win_7_ios['label'] = data_win_7_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=100)&(x.create_role_roi>=0.11)&(x.create_role_pay_cost<=8000) else 1), axis=1)
        result_df = result_df.append(data_win_7_ios[(data_win_7_ios['label'] == 1)| (data_win_7_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_7_ios[data_win_7_ios['label'] == 2])
        
    result_temp = result_df.append(temp_df)
    
    # 结果输出
    if result_temp.shape[0]!=0:
        part_1 = result_temp[result_temp['data_win']==0]
        part_2 = result_temp[result_temp['data_win']==7]
        if part_1.shape[0] != 0:
            temp = result_temp.drop(part_1.index,axis=0)
        else:
            temp = result_temp
        if temp.shape[0] != 0:
            if part_2.shape[0] != 0:
                temp = temp.drop(part_2.index,axis=0)
            # win1,2,3按优先级3,2,1进行去重
            temp.sort_values('data_win',ascending=False,inplace=True)
            temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
            # win1,2,3去重后，与win0进行去重，优先win=0
            if part_1.shape[0] != 0:
                temp = pd.concat([temp,part_1], axis=0)
                temp.sort_values('data_win',inplace=True)
                temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
            # 再去win=7进行去重，win=7优先级小
            if part_2.shape[0] != 0:
                temp = pd.concat([temp,part_2], axis=0)
                temp.sort_values('data_win',inplace=True)
                temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
            source_predict = temp 
            
        elif part_1.shape[0]!=0:
            temp = part_1
            if part_2.shape[0] != 0:
                temp = pd.concat([temp,part_2], axis=0)
                temp.sort_values('data_win',inplace=True)
                temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
            source_predict = temp 
            
        else:
            source_predict = part_2
    else:
        source_predict = pd.DataFrame(columns=["channel_id","source_id","data_win","source_run_datetime","create_time",
                            "media_id","game_id","platform","source_run_date_amount","create_role_num","create_role_cost",
                            "create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi",
                            "create_role_retain_1d","label"])    
    source_predict['label'] = source_predict['label'].astype(int)

    return {"columns": source_predict.columns.values.tolist(), "list": source_predict.values.tolist()}

In [246]:
def source_predict(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"]}
    :return:
    '''
    data = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])
    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]    # 删除消耗为0的数据
    # 去重，处理某天没有消耗的情况
#     if data.shape[0] != 0:
    data.sort_values(by='data_win',inplace=True)
    data.drop_duplicates(subset=['channel_id','source_id','source_run_date_amount','create_role_num'],keep='first'
                           ,inplace=True)
    # 对消耗500元以下的不作判断，对数据按平台、win进行分类处理
    data_ignore = data[data['source_run_date_amount'] < 500]
    data_not_ignore = data[data['source_run_date_amount'] >= 500]
    
    data_an = data_not_ignore[data_not_ignore['platform'] == 1]
    data_ios = data_not_ignore[data_not_ignore['platform'] == 2]
#     if data_an.shape[0] != 0:
    data_win_0 = data_an[data_an['data_win'] == 0]
    data_win_1 = data_an[data_an['data_win'] == 1]
    data_win_2 = data_an[data_an['data_win'] == 2]
    data_win_3 = data_an[data_an['data_win'] == 3]
    data_win_7 = data_an[data_an['data_win'] == 7]
#     else:
#         data_win_0 = pd.DataFrame()
#         data_win_1 = pd.DataFrame()
#         data_win_2 = pd.DataFrame()
#         data_win_3 = pd.DataFrame()
#         data_win_7 = pd.DataFrame()

#     if data_ios.shape[0] != 0:
    data_win_0_ios = data_ios[data_ios['data_win'] == 0]
    data_win_1_ios = data_ios[data_ios['data_win'] == 1]
    data_win_2_ios = data_ios[data_ios['data_win'] == 2]
    data_win_3_ios = data_ios[data_ios['data_win'] == 3]
    data_win_7_ios = data_ios[data_ios['data_win'] == 7]
#     else:
#         data_win_0_ios = pd.DataFrame()
#         data_win_1_ios = pd.DataFrame()
#         data_win_2_ios = pd.DataFrame()
#         data_win_3_ios = pd.DataFrame()
#         data_win_7_ios = pd.DataFrame()  
        
#     if data_not_ignore.shape[0] != 0:
#         data_an = data_not_ignore[data_not_ignore['platform'] == 1]
#         data_ios = data_not_ignore[data_not_ignore['platform'] == 2]
#         if data_an.shape[0] != 0:
#             data_win_0 = data_an[data_an['data_win'] == 0]
#             data_win_1 = data_an[data_an['data_win'] == 1]
#             data_win_2 = data_an[data_an['data_win'] == 2]
#             data_win_3 = data_an[data_an['data_win'] == 3]
#             data_win_7 = data_an[data_an['data_win'] == 7]
#         else:
#             data_win_0 = pd.DataFrame()
#             data_win_1 = pd.DataFrame()
#             data_win_2 = pd.DataFrame()
#             data_win_3 = pd.DataFrame()
#             data_win_7 = pd.DataFrame()
            
#         if data_ios.shape[0] != 0:
#             data_win_0_ios = data_ios[data_ios['data_win'] == 0]
#             data_win_1_ios = data_ios[data_ios['data_win'] == 1]
#             data_win_2_ios = data_ios[data_ios['data_win'] == 2]
#             data_win_3_ios = data_ios[data_ios['data_win'] == 3]
#             data_win_7_ios = data_ios[data_ios['data_win'] == 7]
#         else:
#             data_win_0_ios = pd.DataFrame()
#             data_win_1_ios = pd.DataFrame()
#             data_win_2_ios = pd.DataFrame()
#             data_win_3_ios = pd.DataFrame()
#             data_win_7_ios = pd.DataFrame()
    
    result_df = pd.DataFrame()
    temp_df = pd.DataFrame()
    # win=0预判
    if data_win_0.shape[0] != 0:
        data_win_0['label'] = data_win_0.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_cost>=20)&
            (x.create_role_pay_num==0) else(0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.02) else 
                                            (1 if x.create_role_cost>=50 else 2))),axis=1)
        result_df = result_df.append(data_win_0[(data_win_0['label'] == 1)| (data_win_0['label'] == 0)])
        temp_df = temp_df.append(data_win_0[data_win_0['label'] == 2])
    if data_win_0_ios.shape[0] != 0:    
        data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_cost>=40)&
            (x.create_role_pay_num==0) else(0 if (x.create_role_cost<=300)&(x.create_role_roi>=0.02) else 
                                            (1 if x.create_role_cost>=120 else 2))),axis=1)
        result_df = result_df.append(data_win_0_ios[(data_win_0_ios['label'] == 1)| (data_win_0_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_0_ios[data_win_0_ios['label'] == 2])
        
    # win=1预判
    if data_win_1.shape[0] != 0:
        data_win_1['label'] = data_win_1.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.02)
                                    else (1 if x.create_role_cost>=50 else (1 if x.create_role_pay_sum==0 else 2))),axis=1)
        result_df = result_df.append(data_win_1[(data_win_1['label'] == 1)| (data_win_1['label'] == 0)])
        temp_df = temp_df.append(data_win_1[data_win_1['label'] == 2])
    if data_win_1_ios.shape[0] != 0:
        data_win_1_ios['label'] = data_win_1_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=300)&(x.create_role_roi>=0.02)
                                        else (1 if x.create_role_cost>=120 else (1 if x.create_role_pay_sum==0 else 2))),axis=1)
        result_df = result_df.append(data_win_1_ios[(data_win_1_ios['label'] == 1)| (data_win_1_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_1_ios[data_win_1_ios['label'] == 2])
    
    # win=2预判
    if data_win_2.shape[0] != 0:
        data_win_2['label'] = data_win_2.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.045)
                                    else (1 if x.create_role_cost>=50 else (1 if x.create_role_roi<=0.03 else 2))),axis=1)
        result_df = result_df.append(data_win_2[(data_win_2['label'] == 1)| (data_win_2['label'] == 0)])
        temp_df = temp_df.append(data_win_2[data_win_2['label'] == 2])
        
    if data_win_2_ios.shape[0] != 0:
        data_win_2_ios['label'] = data_win_2_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=250)&(x.create_role_roi>=0.045)
                                        else (1 if x.create_role_cost>=120 else (1 if x.create_role_roi<=0.02 else 2))),axis=1)
        result_df = result_df.append(data_win_2_ios[(data_win_2_ios['label'] == 1)| (data_win_2_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_2_ios[data_win_2_ios['label'] == 2])
   
    # win=3预判
    if data_win_3.shape[0] != 0:
        data_win_3['label'] = data_win_3.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.07)
            else (1 if x.create_role_pay_cost > 5000 else (1 if x.create_role_retain_1d<0.05 else(1 if x.create_role_roi<0.04 else 2))))),axis=1)
        result_df = result_df.append(data_win_3[(data_win_3['label'] == 1)| (data_win_3['label'] == 0)])
        temp_df = temp_df.append(data_win_3[data_win_3['label'] == 2])
    if data_win_3_ios.shape[0] != 0:
        data_win_3_ios['label'] = data_win_3_ios.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (0 if (x.create_role_cost<=250)&(x.create_role_roi>=0.07)
                else (1 if x.create_role_pay_cost >10000 else (1 if x.create_role_retain_1d<0.05 else(1 if x.create_role_roi<0.04 else 2))))),axis=1)
        result_df = result_df.append(data_win_3_ios[(data_win_3_ios['label'] == 1)| (data_win_3_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_3_ios[data_win_3_ios['label'] == 2])
    
    
    # 模型预测
    if temp_df.shape[0] != 0:
        temp_df['label'] = gbdt.predict(temp_df[['data_win','platform','create_role_cost','create_role_pay_cost','create_role_roi']])
    # win=7
    if data_win_7.shape[0] != 0:
        data_win_7['label'] = data_win_7.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=50)&(x.create_role_roi>=0.125)&(x.create_role_pay_cost<=4000) else 1), axis=1)
        result_df = result_df.append(data_win_7[(data_win_7['label'] == 1)| (data_win_7['label'] == 0)])
        temp_df = temp_df.append(data_win_7[data_win_7['label'] == 2])
    if data_win_7_ios.shape[0] != 0:
        data_win_7_ios['label'] = data_win_7_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=100)&(x.create_role_roi>=0.11)&(x.create_role_pay_cost<=8000) else 1), axis=1)
        result_df = result_df.append(data_win_7_ios[(data_win_7_ios['label'] == 1)| (data_win_7_ios['label'] == 0)])
        temp_df = temp_df.append(data_win_7_ios[data_win_7_ios['label'] == 2])
        
    result_temp = result_df.append(temp_df)
    
    # 结果输出
    if result_temp.shape[0]!=0:
        part_1 = result_temp[result_temp['data_win']==0]
        part_2 = result_temp[result_temp['data_win']==7]
        if part_1.shape[0] != 0:
            temp = result_temp.drop(part_1.index,axis=0)
        else:
            temp = result_temp
        if temp.shape[0] != 0:
            if part_2.shape[0] != 0:
                temp = temp.drop(part_2.index,axis=0)
            # win1,2,3按优先级3,2,1进行去重
            temp.sort_values('data_win',ascending=False,inplace=True)
            temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
            # win1,2,3去重后，与win0进行去重，优先win=0
            if part_1.shape[0] != 0:
                temp = pd.concat([temp,part_1], axis=0)
                temp.sort_values('data_win',inplace=True)
                temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
            # 再去win=7进行去重，win=7优先级小
            if part_2.shape[0] != 0:
                temp = pd.concat([temp,part_2], axis=0)
                temp.sort_values('data_win',inplace=True)
                temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
            source_predict = temp 
            
        elif part_1.shape[0]!=0:
            temp = part_1
            if part_2.shape[0] != 0:
                temp = pd.concat([temp,part_2], axis=0)
                temp.sort_values('data_win',inplace=True)
                temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
            source_predict = temp 
            
        else:
            source_predict = part_2
    else:
        source_predict = pd.DataFrame(columns=["channel_id","source_id","data_win","source_run_datetime","create_time",
                            "media_id","game_id","platform","source_run_date_amount","create_role_num","create_role_cost",
                            "create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi",
                            "create_role_retain_1d","label"])    
    source_predict['label'] = source_predict['label'].astype(int)

    return {"columns": source_predict.columns.values.tolist(), "list": source_predict.values.tolist()}

In [126]:
source_predict = pd.DataFrame(columns=["channel_id","source_id","data_win","source_run_datetime","create_time",
                    "media_id","game_id","platform","source_run_date_amount","create_role_num","create_role_cost",
                    "create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi",
                    "create_role_retain_1d","label"])

In [300]:
aa = pd.DataFrame({"A":[1,2,1,1,1,1,1,1],"B":[6,4,6,4,5,4,5,7]})
bb = pd.DataFrame()

In [301]:
def sq(x):
    return x**2

In [310]:
aa['C'] = map(sq,aa['B'])

AttributeError: 'map' object has no attribute 'value'

In [305]:
aa

,A,B,C
0,1,6,<map object at 0x00000000B972C908>
1,2,4,<map object at 0x00000000B972C908>
2,1,6,<map object at 0x00000000B972C908>
3,1,4,<map object at 0x00000000B972C908>
4,1,5,<map object at 0x00000000B972C908>
5,1,4,<map object at 0x00000000B972C908>
6,1,5,<map object at 0x00000000B972C908>
7,1,7,<map object at 0x00000000B972C908>


In [313]:
0.5 == int(1)

True

In [331]:
import math

In [332]:
def is_sqr(x):
    return math.sqrt(x) % 1 == 0

In [333]:
is_sqr(0)

True

In [346]:
from functools import reduce

In [338]:
import math
def is_sqr(x):
    return math.sqrt(x) % 1 == 0
print(list(filter(is_sqr, range(1, 101))))

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [307]:
list(la)

['1', '2', '3', '4', '5']

In [ ]:
def source_predict(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"]}
    :return:
    '''
    data = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])
    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]    # 删除消耗为0的数据
    # 去重，处理某天没有消耗的情况
    if data.shape[0] != 0:
        data.sort_values(by='data_win',inplace=True)
        data.drop_duplicates(subset=['channel_id','source_id','source_run_date_amount','create_role_num'],keep='first'
                               ,inplace=True)
    # 对消耗500元以下的不作判断，对数据按平台、win进行分类处理
#         data_ignore = data[data['source_run_date_amount'] < 500]
        data_not_ignore = data[data['source_run_date_amount'] >= 500]
    data_an = data_not_ignore[data_not_ignore['platform'] == 1]
    data_ios = data_not_ignore[data_not_ignore['platform'] == 2]    
    data_win_0 = data_an[data_an['data_win'] == 0]
    data_win_1 = data_an[data_an['data_win'] == 1]
    data_win_2 = data_an[data_an['data_win'] == 2]
    data_win_3 = data_an[data_an['data_win'] == 3]
    data_win_7 = data_an[data_an['data_win'] == 7]
    data_win_0_ios = data_ios[data_ios['data_win'] == 0]
    data_win_1_ios = data_ios[data_ios['data_win'] == 1]
    data_win_2_ios = data_ios[data_ios['data_win'] == 2]
    data_win_3_ios = data_ios[data_ios['data_win'] == 3]
    data_win_7_ios = data_ios[data_ios['data_win'] == 7]
    
    result_df = pd.DataFrame()
    temp_df = pd.DataFrame()
    # win=0预判
    data_win_0['label'] = data_win_0.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_cost>=20)&
        (x.create_role_pay_num==0) else(0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.02) else 
                                        (1 if x.create_role_cost>=50 else 2))),axis=1)
    data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_cost>=40)&
        (x.create_role_pay_num==0) else(0 if (x.create_role_cost<=300)&(x.create_role_roi>=0.02) else 
                                        (1 if x.create_role_cost>=120 else 2))),axis=1)    
    result_df = result_df.append(data_win_0[(data_win_0['label'] == 1)| (data_win_0['label'] == 0)]).append(data_win_0_ios[(data_win_0_ios['label'] == 1)|(data_win_0_ios['label'] == 0)])
    temp_df = temp_df.append(data_win_0[data_win_0['label'] == 2]).append(data_win_0_ios[data_win_0_ios['label'] == 2])
    # win=1预判
    data_win_1['label'] = data_win_1.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.02)
                                else (1 if x.create_role_cost>=50 else (1 if x.create_role_pay_sum==0 else 2))),axis=1)
    data_win_1_ios['label'] = data_win_1_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=300)&(x.create_role_roi>=0.02)
                                    else (1 if x.create_role_cost>=120 else (1 if x.create_role_pay_sum==0 else 2))),axis=1)
    result_df = result_df.append(data_win_1[(data_win_1['label'] == 1)| (data_win_1['label'] == 0)]).append(data_win_1_ios[(data_win_1_ios['label'] == 1)|(data_win_1_ios['label'] == 0)])
    temp_df = temp_df.append(data_win_1[data_win_1['label'] == 2]).append(data_win_1_ios[data_win_1_ios['label'] == 2])
    # win=2预判
    data_win_2['label'] = data_win_2.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.045)
                                else (1 if x.create_role_cost>=50 else (1 if x.create_role_roi<=0.03 else 2))),axis=1)
    data_win_2_ios['label'] = data_win_2_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=250)&(x.create_role_roi>=0.045)
                                    else (1 if x.create_role_cost>=120 else (1 if x.create_role_roi<=0.02 else 2))),axis=1)
    result_df = result_df.append(data_win_2[(data_win_2['label'] == 1)| (data_win_2['label'] == 0)]).append(data_win_2_ios[(data_win_2_ios['label'] == 1)|(data_win_2_ios['label'] == 0)])
    temp_df = temp_df.append(data_win_2[data_win_2['label'] == 2]).append(data_win_2_ios[data_win_2_ios['label'] == 2])
    # win=3预判
    data_win_3['label'] = data_win_3.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.07)
        else (1 if x.create_role_pay_cost > 5000 else (1 if x.create_role_retain_1d<0.05 else(1 if x.create_role_roi<0.04 else 2))))),axis=1)
    data_win_3_ios['label'] = data_win_3_ios.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (0 if (x.create_role_cost<=250)&(x.create_role_roi>=0.07)
            else (1 if x.create_role_pay_cost >10000 else (1 if x.create_role_retain_1d<0.05 else(1 if x.create_role_roi<0.04 else 2))))),axis=1)
    result_df = result_df.append(data_win_3[(data_win_3['label'] == 1)| (data_win_3['label'] == 0)]).append(data_win_3_ios[(data_win_3_ios['label'] == 1)|(data_win_3_ios['label'] == 0)])
    temp_df = temp_df.append(data_win_3[data_win_3['label'] == 2]).append(data_win_3_ios[data_win_3_ios['label'] == 2])
    
    # 模型预测
    temp_df['label'] = gbdt.predict(temp_df[['data_win','platform','create_role_cost','create_role_pay_cost','create_role_roi']])
    # win=7
    data_win_7['label'] = data_win_7.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=50)&(x.create_role_roi>=0.125)&(x.create_role_pay_cost<=4000) else 1), axis=1)
    data_win_7_ios['label'] = data_win_7_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=100)&(x.create_role_roi>=0.11)&(x.create_role_pay_cost<=8000) else 1), axis=1)
    
    result_df = result_df.append(data_win_7).append(data_win_7_ios)
    result_temp = pd.concat([result_df,temp_df],axis=0)
    
    # 结果输出
    part_1 = result_temp[result_temp['data_win']==0]
    part_2 = result_temp[result_temp['data_win']==7]
    temp = result_temp.drop(part_1.index,axis=0)
    temp = temp.drop(part_2.index,axis=0)
    
    # 按优先级顺序输出去重后的预测结果
    temp.sort_values('data_win',ascending=False,inplace=True)
    temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
    
    temp = pd.concat([temp,part_1], axis=0)
    temp.sort_values('data_win',inplace=True)
    temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)
    
    temp = pd.concat([temp,part_2], axis=0)
    temp.sort_values('data_win',inplace=True)
    source_predict = temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first')
    
    source_predict['label'] = source_predict['label'].astype(int)

    return {"columns": source_predict.columns.values.tolist(), "list": source_predict.values.tolist()}